In [ ]:
# install packages
!pip install catboost
!pip install xgboost
!pip install pycaret[full]

In [ ]:
# install lightgbm GPU in colab
# 先登入google cloud
from google.colab import drive
drive.mount('/content/drive')

!pip uninstall lightgbm -y
!git clone --recursive https://github.com/Microsoft/LightGBM
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu

In [1]:
# import packages
import os
import numpy as np
import pandas as pd

# Pycaret
from pycaret.classification import *

import catboost
import lightgbm
import xgboost

from sklearn.preprocessing import LabelEncoder
from joblib import load, dump

def check_gpu_support():
    try:
        data = np.random.rand(50, 2)
        label = np.random.randint(2, size=50)
        train_data = lightgbm.Dataset(data, label=label)
        params = {'device': 'gpu'}
        gbm = lightgbm.train(params, train_set=train_data)
        return True
    except Exception as e:
        return False

print(check_gpu_support())

C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


False


In [2]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
path = "/content/drive/My Drive/colab/TPS May"
# path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - May 2021'
os.chdir(path)

In [3]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [4]:
num_col = []
cat_col = [i for i in df_train.columns if i not in ['id', 'target']]
target_col = 'target'
comb = num_col + cat_col + [target_col]

In [11]:
# Set-Up
setup(data = df_train[comb],
      categorical_features=cat_col,
      numeric_features=num_col,
      imputation_type='iterative',
      target = target_col,
      train_size=0.8,
      fold = 5,
      # folds_shuffle = False,   # exist error
      n_jobs=-1,
      use_gpu = True,
      session_id=42,   # seed
      silent = True)

,Description,Value
0,session_id,42
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,"Class_1: 0, Class_2: 1, Class_3: 2, Class_4: 3"
4,Original Data,"(100000, 51)"
5,Missing Values,False
6,Numeric Features,0
7,Categorical Features,50
8,Ordinal Features,False
9,High Cardinality Features,False


(-1,
 -1,
 {'USI',
  'X',
  'X_test',
  'X_train',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'data_before_preprocess',
  'display_container',
  'exp_name_log',
  'experiment__',
  'fix_imbalance_method_param',
  'fix_imbalance_param',
  'fold_generator',
  'fold_groups_param',
  'fold_groups_param_full',
  'fold_param',
  'fold_shuffle_param',
  'gpu_param',
  'html_param',
  'imputation_classifier',
  'imputation_regressor',
  'iterative_imputation_iters_param',
  'log_plots_param',
  'logging_param',
  'master_model_container',
  'n_jobs_param',
  'prep_pipe',
  'pycaret_globals',
  'seed',
  'stratify_param',
  'target_param',
  'transform_target_method_param',
  'transform_target_param',
  'y',
  'y_test',
  'y_train'},
 Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['feature_0',
      

In [ ]:
# GPU Enable
models(internal=True)[['Name', 'GPU Enabled']]

In [ ]:
# check all metrics used for model evaluation
get_metrics()

# add Log Loss metric in pycaret
from sklearn.metrics import log_loss
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)

In [ ]:
TOP_10 = compare_models(sort='logloss',   # or ['Accuracy', 'AUC', 'Recall', 'Precision', 'F1', 'Kappa', 'MCC']
                        n_select=10, 
                        budget_time=1*60)   # in minutes(?)

IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:51:47
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.5763,0.5879,0.2672,0.4562,0.4541,0.0478,0.0864,128.1080


In [13]:
# select best model 
best = automl(optimize = 'AUC')

ValueError: max() arg is an empty sequence